#Requirements

In [1]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
from google.colab import drive
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from datasets import load_dataset
from torch.utils.data import DataLoader
from torch import cuda, tensor, no_grad
import torch
from tqdm.auto import tqdm
import json

In [3]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


#Model Initialization

In [4]:
device = 'cuda' if cuda.is_available() else 'cpu'
pipe = pipeline(
    "text-generation",
    model="openai-community/gpt2-large",
    torch_dtype=torch.bfloat16,
    device_map="auto",
)

config.json:   0%|          | 0.00/666 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.25G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [5]:
indiv_rubric = '''assigns a holistic score for the essay based on the rubric below. For the following evaluations you will need to use a grading scale between 1 (minimum) and 6 (maximum). As with the analytical rating form, the distance between each grade (e.g., 1-2, 3-4, 4-5) should be considered equal.
SCORE OF 6: An essay in this category demonstrates clear and consistent mastery, although it may have a few minor errors. A typical essay effectively and insightfully develops a point of view on the issue and demonstrates outstanding critical thinking, using clearly appropriate examples, reasons, and other evidence to support its position; the essay is well organized and clearly focused, demonstrating clear coherence and smooth progression of ideas; the essay exhibits skillful use of language, using a varied, accurate, and apt vocabulary and demonstrates meaningful variety in sentence structure; the essay is free of most errors in grammar, usage, and mechanics.
SCORE OF 5: An essay in this category demonstrates reasonably consistent mastery, although it will have occasional errors or lapses in quality. A typical essay effectively develops a point of view on the issue and demonstrates strong critical thinking, generally using appropriate examples, reasons, and other evidence to support its position; the essay is well organized and focused, demonstrating coherence and progression of ideas; the essay exhibits facility in the use of language, using appropriate vocabulary demonstrates variety in sentence structure; the essay is generally free of most errors in grammar, usage, and mechanics.
SCORE OF 4: An essay in this category demonstrates adequate mastery, although it will have lapses in quality. A typical essay develops a point of view on the issue and demonstrates competent critical thinking, using adequate examples, reasons, and other evidence to support its position; the essay is generally organized and focused, demonstrating some coherence and progression of ideas exhibits adequate; the essay may demonstrate inconsistent facility in the use of language, using generally appropriate vocabulary demonstrates some variety in sentence structure; the essay may have some errors in grammar, usage, and mechanics.
SCORE OF 3: An essay in this category demonstrates developing mastery, and is marked by ONE OR MORE of the following weaknesses: develops a point of view on the issue, demonstrating some critical thinking, but may do so inconsistently or use inadequate examples, reasons, or other evidence to support its position; the essay is limited in its organization or focus, or may demonstrate some lapses in coherence or progression of ideas displays; the essay may demonstrate facility in the use of language, but sometimes uses weak vocabulary or inappropriate word choice and/or lacks variety or demonstrates problems in sentence structure; the essay may contain an accumulation of errors in grammar, usage, and mechanics.
SCORE OF 2: An essay in this category demonstrates little mastery, and is flawed by ONE OR MORE of the following weaknesses: develops a point of view on the issue that is vague or seriously limited, and demonstrates weak critical thinking, providing inappropriate or insufficient examples, reasons, or other evidence to support its position; the essay is poorly organized and/or focused, or demonstrates serious problems with coherence or progression of ideas; the essay displays very little facility in the use of language, using very limited vocabulary or incorrect word choice and/or demonstrates frequent problems in sentence structure; the essay contains errors in grammar, usage, and mechanics so serious that meaning is somewhat obscured.
SCORE OF 1: An essay in this category demonstrates very little or no mastery, and is severely flawed by ONE OR MORE of the following weaknesses: develops no viable point of view on the issue, or provides little or no evidence to support its position; the essay is disorganized or unfocused, resulting in a disjointed or incoherent essay; the essay displays fundamental errors in vocabulary and/or demonstrates severe flaws in sentence structure; the essay contains pervasive errors in grammar, usage, or mechanics that persistently interfere with meaning.'''

#Dataset Loading and Input Formatting: DEFAULT

In [6]:
def preprocess(data):
    batch = {}
    batch['messages'] = [f"Based on the following rubric:\n'{indiv_rubric}'\nAnd the following prompt:\n'{prompt}'\nThe essay:\n'{full_text}'.\ndeserves a score of "
    for full_text, prompt in zip(data['full_text'], data['assignment'])]
    batch['labels'] = data['holistic_essay_score']
    batch['essay_ids'] = data['essay_id']
    return batch

def compute_metric():
    return

def collate(data):
    return {'messages': [ex['messages'] for ex in data],
            'labels': [ex['labels'] for ex in data],
            'essay_ids': [ex['essay_ids'] for ex in data]}

In [7]:
batch_size = 1
print('loading dataset...\n')
train_data = load_dataset('json', data_files={'train':'/content/drive/MyDrive/COS597H/data/train_indiv.json'})
train_data = train_data['train'].select_columns(['full_text', 'assignment', 'holistic_essay_score', 'essay_id'])
train_data = train_data.map(preprocess, batched=True, batch_size=batch_size)
train_dataloader = DataLoader(train_data,
                               batch_size=batch_size, shuffle=False,
                               collate_fn=collate)

loading dataset...



Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/7868 [00:00<?, ? examples/s]

#Inference: DEFAULT

In [23]:
epochs = 1

num_training_steps = epochs * len(train_dataloader)
progress_bar = tqdm(range(num_training_steps))

with open ('/content/drive/MyDrive/COS597H/outputs/raw_gpt_reg.csv', 'w') as out_file:
    out_file.write('essay_id,pred\n')
    with no_grad():
        for _ in range(epochs):
            for i, batch in enumerate(train_dataloader):
                essay_ids = batch['essay_ids']
                for i in range(len(batch['messages'])):
                    out = pipe(batch['messages'][i], max_length=55, max_new_tokens = 20,pad_token_id=pipe.tokenizer.eos_token_id)
                    #out = pipe(batch['messages'], max_new_tokens=1, pad_token_id=pipe.tokenizer.eos_token_id
                    out_file.write(f'{essay_ids[i]},"{out[i][0]["generated_text"]}"\n')
                    print(f'{essay_ids[i]},"{out[i][0]["generated_text"]}"')
                progress_bar.update()

  0%|          | 0/7868 [00:00<?, ?it/s]

Both `max_new_tokens` (=20) and `max_length`(=55) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


KeyError: 0

#Dataset Loading and Input Formatting: Demographics

In [ ]:
def preprocess(data):
    demos = []
    for var, val in zip(data['demographic_variable'], data['demographic_value']):
        match var:
            case "gender":
                demos.append("male" if val == "M" else "female")
            case "grade_level":
                demos.append(f"in grade {val}")
            case "ell_status":
                demos.append("an English Language Learner" if val == "Yes" else "fluent in English")
            case "race_ethnicity":
                demos.append(val)
            case _:
                demos.append(val.lower())
    batch = {}
    batch['messages'] = [f"Consider a student's essay response. The student is {demo}.\nBased on the following rubric:\n'{indiv_rubric}'\nAnd the following prompt:\n'{prompt}'\nThe student's essay:\n'{full_text}'.\ndeserves a score of "
    for full_text, prompt, demo in zip(data['full_text'], data['assignment'], demos)]
    batch['labels'] = data['holistic_essay_score']
    batch['essay_ids'] = data['essay_id']
    batch['demographic_value'] = data['demographic_value']
    return batch

def compute_metric():
    return

def collate(data):
    return {'messages': [ex['messages'] for ex in data],
            'labels': [ex['labels'] for ex in data],
            'essay_ids': [ex['essay_ids'] for ex in data],
            'demographic_value': [ex['demographic_value'] for ex in data]}

#Inference: Demographics



In [ ]:
batch_size = 8
print('loading dataset...\n')
train_data = load_dataset('json', data_files={'train':'/content/drive/MyDrive/COS597H/data/train_indiv_demo_shuffled2.json'})
train_data = train_data['train'].select_columns(['full_text', 'assignment', 'holistic_essay_score', 'essay_id', 'demographic_variable', 'demographic_value'])
train_data = train_data.map(preprocess, batched=True, batch_size=batch_size)
train_dataloader = DataLoader(train_data,
                               batch_size=batch_size, shuffle=False,
                               collate_fn=collate)

In [ ]:
epochs = 1

num_training_steps = epochs * len(train_dataloader)
progress_bar = tqdm(range(num_training_steps))

with open ('/content/drive/MyDrive/COS597H/outputs/raw_gpt_demo.csv', 'w') as out_file:
    out_file.write('essay_id,demographic_value,pred\n')
    with no_grad():
        for _ in range(epochs):
            for i, batch in enumerate(train_dataloader):
                essay_ids = batch['essay_ids']
                cf_values = batch['demographic_value']
                out = pipe(batch['messages'], max_new_tokens=1, pad_token_id=pipe.tokenizer.eos_token_id)
                for i in range(len(batch['messages'])):
                    #out = pipe(batch['messages'], max_new_tokens=1, pad_token_id=pipe.tokenizer.eos_token_id)
                    out_file.write(f'{essay_ids[i]},"{cf_values[i]}","{out[i][0]["generated_text"][-1]}"\n')
                    #print(f'{essay_ids[i]},"{out[i][0]["generated_text"][-1]}"')
                progress_bar.update()

In [ ]:
batch_size = 8
print('loading dataset...\n')
train_data = load_dataset('json', data_files={'train':'/content/drive/MyDrive/COS597H/data/train_indiv_demo_shuffled_cf2.json'})
train_data = train_data['train'].select_columns(['full_text', 'assignment', 'holistic_essay_score', 'essay_id', 'demographic_variable', 'demographic_value'])
train_data = train_data.map(preprocess, batched=True, batch_size=batch_size)
train_dataloader = DataLoader(train_data,
                               batch_size=batch_size, shuffle=False,
                               collate_fn=collate)

In [ ]:
epochs = 1

num_training_steps = epochs * len(train_dataloader)
progress_bar = tqdm(range(num_training_steps))

with open ('/content/drive/MyDrive/COS597H/outputs/raw_gpt_demo_cf.csv', 'w') as out_file:
    out_file.write('essay_id,demographic_value,pred\n')
    with no_grad():
        for _ in range(epochs):
            for i, batch in enumerate(train_dataloader):
                essay_ids = batch['essay_ids']
                cf_values = batch['demographic_value']
                out = pipe(batch['messages'], max_new_tokens=1, pad_token_id=pipe.tokenizer.eos_token_id)
                for i in range(len(batch['messages'])):
                    #out = pipe(batch['messages'], max_new_tokens=1, pad_token_id=pipe.tokenizer.eos_token_id)
                    out_file.write(f'{essay_ids[i]},"{cf_values[i]}","{out[i][0]["generated_text"][-1]}"\n')
                    #print(f'{essay_ids[i]},"{out[i][0]["generated_text"][-1]}"')
                progress_bar.update()

#output post-processing

In [ ]:
import pandas as pd
import re

def extract_integers(csv_path, output_path):
    df = pd.read_csv(csv_path)
    # print(df.head())
    df.iloc[:, 2] = df.iloc[:, 2].astype(str).apply(
        lambda x: ''.join(re.findall(r'\d+', x))
    )

    df.to_csv(output_path, index=False)

In [ ]:
extract_integers('/content/drive/MyDrive/COS597H/outputs/raw_gpt_reg.csv',
                '/content/drive/MyDrive/COS597H/outputs/gpt_reg.csv')

In [ ]:
extract_integers('/content/drive/MyDrive/COS597H/outputs/raw_gpt_demo.csv',
                '/content/drive/MyDrive/COS597H/outputs/gpt_demo.csv')

In [ ]:
extract_integers('/content/drive/MyDrive/COS597H/outputs/raw_gpt_demo_cf.csv',
                '/content/drive/MyDrive/COS597H/outputs/gpt_demo_cf.csv')